In [0]:
sc

<SparkContext master=spark://10.139.64.4:7077 appName=Databricks Shell>

In [0]:
spark

In [0]:
shdf=spark.read.csv("/Volumes/azuredatabricks/default/shopping/Online Shoppers Intention.csv",header=True,inferSchema=True)

In [0]:
shdf.printSchema()

root
 |-- Administrative: integer (nullable = true)
 |-- Administrative_Duration: double (nullable = true)
 |-- Informational: integer (nullable = true)
 |-- Informational_Duration: double (nullable = true)
 |-- ProductRelated: integer (nullable = true)
 |-- ProductRelated_Duration: double (nullable = true)
 |-- BounceRates: double (nullable = true)
 |-- ExitRates: double (nullable = true)
 |-- PageValues: double (nullable = true)
 |-- SpecialDay: double (nullable = true)
 |-- Month: string (nullable = true)
 |-- OperatingSystems: integer (nullable = true)
 |-- Browser: integer (nullable = true)
 |-- Region: integer (nullable = true)
 |-- TrafficType: integer (nullable = true)
 |-- VisitorType: string (nullable = true)
 |-- Weekend: boolean (nullable = true)
 |-- Revenue: boolean (nullable = true)



data cleaning and preprocessing

In [0]:
from pyspark.sql.functions import*
shdf.select([count(when(isnull(col), col)).alias(col) for col in shdf.columns]).show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|Month|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|Revenue|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|            14|                     14|           14|                    14|            14|                     14|         14|       14|         0|         0|    0|               0|      0|     0|          0|          0|      0|      0|
+--------------+-----------------------+----

In [0]:
shdf=shdf.na.drop()

In [0]:
shdf=shdf.dropDuplicates().fillna

In [0]:
shdf.select([count(when(isnull(col), col)).alias(col) for col in shdf.columns]).show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|Month|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|Revenue|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|             0|                      0|            0|                     0|             0|                      0|          0|        0|         0|         0|    0|               0|      0|     0|          0|          0|      0|      0|
+--------------+-----------------------+----

transformation


In [0]:
#categorical values to numerical

In [0]:
shdf.select('VisitorType').distinct().show()

+-----------------+
|      VisitorType|
+-----------------+
|      New_Visitor|
|            Other|
|Returning_Visitor|
+-----------------+



In [0]:
shdf=shdf.withColumn("VisitorType",regexp_replace("VisitorType","New_Visitor","0"))
shdf=shdf.withColumn("VisitorType",regexp_replace("VisitorType","Other","1"))
shdf=shdf.withColumn("VisitorType",regexp_replace("VisitorType","Returning_Visitor","2"))

In [0]:
from pyspark.sql.types import IntegerType
shdf=shdf.withColumn("VisitorType",col("VisitorType").cast(IntegerType()))

In [0]:
shdf.show(5)

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|  ExitRates| PageValues|SpecialDay|Month|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|Revenue|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|             2|                    2.0|            3|                 261.0|            10|                  281.0|        0.0|0.042857143|        0.0|       0.0|  Mar|               2|      2|     1|          2|          2|  false|  false|
|             0|                

In [0]:
shdf1=shdf.groupby("VisitorType")

In [0]:
shdf1=shdf.groupby("VisitorType").agg(mean("ExitRates").alias("avg exit rate")).show()

+-----------+--------------------+
|VisitorType|       avg exit rate|
+-----------+--------------------+
|          1|0.056600458148148125|
|          2| 0.04477715327692973|
|          0|0.020574603134081524|
+-----------+--------------------+



In [0]:
shdf.orderBy(col("ProductRelated_Duration")).show(10)

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|Month|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|Revenue|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+-----+----------------+-------+------+-----------+-----------+-------+-------+
|             0|                   -1.0|            0|                  -1.0|             1|                   -1.0|        0.2|      0.2|       0.0|       0.0|  Feb|               3|      3|     1|          3|          2|  false|  false|
|             0|                   -1.0|    

In [0]:
partitioned_df=shdf.repartition(3)

In [0]:
partitioned_df.write.mode("append").parquet("/Volumes/azuredatabricks/default/Shopping/")

In [0]:
shdf.cache()

DataFrame[Administrative: int, Administrative_Duration: double, Informational: int, Informational_Duration: double, ProductRelated: int, ProductRelated_Duration: double, BounceRates: double, ExitRates: double, PageValues: double, SpecialDay: double, Month: string, OperatingSystems: int, Browser: int, Region: int, TrafficType: int, VisitorType: int, Weekend: boolean, Revenue: boolean]

joining and merging datasets

In [0]:
flights_df = spark.read.csv("/Volumes/azuredatabricks/default/shopping/raw_flight_data.csv", inferSchema=True, header = True)

In [0]:
airports_df = spark.read.csv("/Volumes/azuredatabricks/default/shopping/airports.csv", header = True,
                            inferSchema=True)

In [0]:
flightairportDF=flights_df.join(airports_df,flights_df.OriginAirportID == airports_df.airport_id)